In [1]:
from scipy.stats import pearsonr, entropy
import numpy as np
import pandas as pd
import akshare as ak
import baostock as bs
from utils import get_index_components, get_csindex_components, get_history_k
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',100)

In [20]:
kc50_list = get_csindex_components("000688")

In [108]:
bs.login()
output_list = []
for stk in kc50_list:
    rs = bs.query_history_k_data_plus(stk,
        "date,time,code,open,high,low,close,volume,amount,adjustflag",
        start_date='2024-12-01', end_date='2024-12-30',
        frequency="60", adjustflag="1")

    data_list = []
    while (rs.error_code == '0') & rs.next():
        data_list.append(rs.get_row_data())
    output_list.append(pd.DataFrame(data_list, columns=rs.fields))
df_output=pd.concat(output_list)
    
bs.logout()

login success!
logout success!


In [107]:
2

0.0378

In [106]:
df_output['datetime'] = pd.to_datetime(df_output['time'].str[:14], format='%Y%m%d%H%M%S')
df_output['hms'] = df_output['datetime'].dt.time.astype(str)

df_manpl = df_output.groupby('datetime').apply(lambda df: pd.Series({"close":df['close'].astype(float).mean(), "hms":df['hms'].values[0]}))
df_manpl = df_manpl[df_manpl['hms'].isin(["11:30:00","15:00:00"])]

df_manpl['return_factor'] = (df_manpl['close']/df_manpl['close'].shift(1)).fillna(1)
df_manpl[df_manpl['hms']=='11:30:00']['return_factor'].product()

1.1455615772832217